In [1]:
import os, cv2, tqdm, torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from PIL import Image
from torchvision import transforms
import numpy as np
import pandas as pd
from shutil import copyfile

# parameters
# model_type = ['ResNet', 'reg']
# model_type = ['ResNet', 'class']
# model_type = ['EfficientNet', 'reg']
model_type = ['EfficientNet', 'class']
pkl_path = '/kaggle/input/iiiiiiiiiiiiii/EffNet_class_weight_9_9127.pt'
pred_path = '/kaggle/working/'
data_path = '/kaggle/input/aptos2019-blindness-detection/'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
img_size = 256
batch = 64
n_cpu = 12
coef = [0.57, 1.37, 2.3, 3.57]  # for regression
print(model_type, device)

['EfficientNet', 'class'] cuda:0


In [2]:
# copy our file into the working directory
os.makedirs('../working/efficientnet/', exist_ok=True)
copyfile(src = "../input/efficientnet-pytorch/model.py", dst = "../working/efficientnet/model.py")
copyfile(src = "../input/efficientnet-pytorch/utils.py", dst = "../working/efficientnet/utils.py")
copyfile(src = "../input/efficientnet-pytorch/__init__.py", dst = "../working/efficientnet/__init__.py")

from efficientnet.model import EfficientNet

In [3]:
# data process class
class dataLoader(data.Dataset):
    def __init__(self, src_path: str, mode: str, img_size: int):
        """
        :param src_path (string): Source path of the dataset
        :param mode (string): Indicate procedure status (train/test)
        :param img_size (int): Image size put into the model
        """
        self.src_path = src_path
        self.mode = mode
        self.img_size = img_size
        self.img_name, self.label = self.get_data()

        if self.mode == 'train':
            self.trans = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                             transforms.RandomVerticalFlip(p=0.5),
                                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                  std=[0.229, 0.224, 0.225])])

        else:
            self.trans = transforms.Compose([transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                  std=[0.229, 0.224, 0.225])])

        print("> {}: Found {} images".format(self.mode, len(self.img_name)))

    def __len__(self):
        """return the size of dataset"""
        return len(self.img_name)

    def __getitem__(self, index):
        """
            return processed image, targets, image id, and image path
        """
        path = self.src_path + self.mode + '_images/' + self.img_name[index] + '.png'
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = crop_image_from_gray(img)
        img = cv2.resize(img, (self.img_size, self.img_size))
        img = cv2.addWeighted (img, 4, cv2.GaussianBlur(img, (0,0) , 30) ,-4 ,128)
        img = transforms.ToTensor()(img)
        img = self.trans(img)

        return img, self.label[index], self.img_name[index]


    def get_data(self):
        """
            return the img_name & label list
        """
        data = pd.read_csv(self.src_path + self.mode + '.csv')
    
        if self.mode == 'train':
            return np.squeeze(data['id_code'].values), np.squeeze(data['diagnosis'].values)
        else:
            return np.squeeze(data['id_code'].values), np.squeeze(data['id_code'].values)

        
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

In [4]:
# dataloader
test_data = dataLoader(data_path, 'test', img_size)
test_len = len(test_data)
test_loader = data.DataLoader(
    dataset=test_data,
    batch_size=batch,
    shuffle=False,
    num_workers=n_cpu
)

> test: Found 1928 images


In [5]:
# build model
n_out = 1 if model_type[1] == 'reg' else 5

if model_type[0] == 'EfficientNet':
    model = EfficientNet.from_name('efficientnet-b5')
    feature = model._fc.in_features
    model._fc = nn.Linear(in_features=feature, out_features=n_out, bias=True)
else:
    model = torchvision.models.resnet50(pretrained=False)
    feature = model.fc.in_features
    model.fc = nn.Linear(in_features=feature, out_features=n_out, bias=True)

model.load_state_dict(torch.load(pkl_path, map_location=device))
model = model.to(device)
print(model)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [7]:
# testing
model.eval()
test_df = pd.DataFrame([], columns=['id_code', 'diagnosis'])
with torch.no_grad():
    for i, (imgs, _, img_name) in enumerate(tqdm.tqdm(test_loader)):        
        if model_type[1] == 'reg':
            imgs = imgs.to(device)
            output = model(imgs)
            output = output.detach().cpu().squeeze().numpy().reshape(-1)

            bound_output = np.zeros(output.shape[0])
            bound_output[np.where(output < coef[0])] = 0
            bound_output[np.where(coef[0] <= output)] = 1
            bound_output[np.where(coef[1] <= output)] = 2
            bound_output[np.where(coef[2] <= output)] = 3
            bound_output[np.where(coef[3] <= output)] = 4

            row_dict = {}
            row_dict['id_code'] = img_name
            row_dict['diagnosis'] = np.squeeze(bound_output).astype(int)
            test_df = pd.concat([test_df, pd.DataFrame.from_dict(row_dict, orient='columns')])
        else:
            imgs = imgs.to(device)
            output = model(imgs)
            highest_out = torch.max(output, 1)[1].cpu()

            row_dict = {}
            row_dict['id_code'] = img_name
            row_dict['diagnosis'] = highest_out
            test_df = pd.concat([test_df, pd.DataFrame.from_dict(row_dict, orient='columns')])
                                                                 
test_df.to_csv('submission.csv', index=False)

100%|██████████| 31/31 [05:39<00:00, 10.96s/it]  
